# DEPENDENCY INSTALLATION

In [ ]:
!pip install -q "transformers>=4.35.0" "accelerate>=0.24.0"
!pip install -q torch torchaudio sentencepiece protobuf pandas

print("✅ Dependencies installed.")

# IMPORTS  

In [ ]:
import gc
import os
import re
import time
import unicodedata
import warnings
from pathlib import Path
from typing import Dict, List, Tuple


import pandas as pd
import torch
import torchaudio
from tqdm.auto import tqdm
from transformers import WhisperForConditionalGeneration, WhisperProcessor

from datetime import datetime
from zoneinfo import ZoneInfo

warnings.filterwarnings("ignore")
print("✅ Imports ready.")

# CONFIGURATION

- All runtime parameters are defined here. Edit before running.

In [ ]:
# ── Model ─────────────────────────────────────────────────────────────────────
MERGED_MODEL = "bitwisemind/sam_15000_clean_text_full_model"

# ── Paths ─────────────────────────────────────────────────────────────────────
AUDIO_BASE_PATH = (
    "/kaggle/input/competitions/"
    "dl-sprint-4-0-bengali-long-form-speech-recognition/"
    "transcription/transcription/test/audio"
)
OUTPUT_CSV = "/kaggle/working/transcriptions_optimized.csv"

# ── Audio File Selection ──────────────────────────────────────────────────────
# List specific filenames to process, e.g. ["test_001.wav", "test_002.wav"].
# Leave empty to process all audio files found in AUDIO_BASE_PATH.
AUDIO_FILES: List[str] = []

# ── Chunking & Batching ───────────────────────────────────────────────────────
CHUNK_LENGTH_S = 25
OVERLAP_S      = 0
BATCH_SIZE     = 8

# ── Optimization ──────────────────────────────────────────────────────────────
USE_STATIC_CACHE  = True   # Enable static KV cache (faster generation on CUDA)
USE_TORCH_COMPILE = True

# ── Device ────────────────────────────────────────────────────────────────────
DEVICE   = "cuda" if torch.cuda.is_available() else "cpu"
USE_FP16 = DEVICE == "cuda"   # fp16 only on GPU

print("✅ Configuration loaded.")
print(f"   Model        : {MERGED_MODEL}")
print(f"   Device       : {DEVICE}  |  FP16 : {USE_FP16}")
print(f"   Chunk / Batch: {CHUNK_LENGTH_S}s / {BATCH_SIZE}")
print(f"   Static cache : {USE_STATIC_CACHE}  |  torch.compile : {USE_TORCH_COMPILE}")

# AUDIO UTILITIES

In [ ]:
def format_time(seconds: float) -> str:
    """Convert a duration in seconds to a human-readable h/m/s string."""
    h, rem = divmod(int(seconds), 3600)
    m, s   = divmod(rem, 60)
    if h > 0:   return f"{h}h {m}m {s}s"
    if m > 0:   return f"{m}m {s}s"
    return f"{s}s"


def load_and_resample_audio(audio_path: str) -> Tuple[torch.Tensor, int]:
    """Load an audio file, convert to mono, and resample to 16 kHz.

    Returns:
        waveform (torch.Tensor): 1-D float32 tensor.
        sample_rate (int): Always 16000 after resampling.
    """
    waveform, sr = torchaudio.load(audio_path)
    waveform = waveform.float()

    # Mix down to mono
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    # Resample to 16 kHz (Whisper requirement)
    if sr != 16000:
        waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
        sr = 16000

    return waveform.squeeze(), sr


def chunk_audio(
    audio: torch.Tensor,
    sample_rate: int,
    chunk_length_s: int,
    overlap_s: float,
) -> List[torch.Tensor]:
    """Split a 1-D audio tensor into fixed-length, optionally overlapping chunks.

    The final chunk is zero-padded to `chunk_length_s` so every chunk has
    the same length (required for batch processing with Whisper).
    """
    chunk_samples   = int(chunk_length_s * sample_rate)
    overlap_samples = int(overlap_s      * sample_rate)
    stride          = chunk_samples - overlap_samples
    chunks, start   = [], 0

    while start < len(audio):
        end   = min(start + chunk_samples, len(audio))
        chunk = audio[start:end]

        # Pad the last chunk so all chunks share the same length
        if len(chunk) < chunk_samples:
            chunk = torch.nn.functional.pad(chunk, (0, chunk_samples - len(chunk)))

        chunks.append(chunk)
        if end >= len(audio):
            break
        start += stride

    return chunks


def merge_transcriptions(transcriptions: List[str]) -> str:
    """Join per-chunk transcriptions into a single string and collapse whitespace."""
    if not transcriptions:
        return ""
    return " ".join(" ".join(transcriptions).split())


def validate_audio_files(
    audio_files: List[str], base_path: str
) -> Tuple[List[Path], List[str]]:
    """Check which requested audio files exist on disk.

    Returns:
        valid_paths  : List of existing Path objects.
        missing_files: List of filenames not found.
    """
    base = Path(base_path)
    valid, missing = [], []
    for f in audio_files:
        p = base / f
        (valid if p.exists() else missing).append(p if p.exists() else f)
    return valid, missing


print("✅ Audio utilities ready.")

# TEXT NORMALIZATION & POST-PROCESSING

In [ ]:
# Zero-width Unicode characters frequently introduced by Bengali ASR outputs
_ZW_PATTERN = re.compile(r"[\u200B-\u200D\uFEFF]")


def normalize_bn_text(text: str) -> str:
    """Normalize Bengali text: NFC unicode, remove zero-width chars, collapse spaces."""
    if not text:
        return ""
    text = unicodedata.normalize("NFC", str(text))
    text = _ZW_PATTERN.sub("", text)
    text = text.replace("\u00A0", " ")   # Non-breaking space → regular space
    return " ".join(text.split())


def clean_transcript(text: str) -> str:
    """Remove ASR hallucination artifacts from a transcript.

    Handles three hallucination patterns common in Whisper outputs:
      1. Multi-word phrases repeated 3+ times in sequence.
      2. Single words repeated 3+ times in sequence.
      3. Character n-grams repeated without spaces (e.g., "হেহেহেহে").
    Also strips '>>' speaker-change markers.
    """
    if not text or (isinstance(text, float) and pd.isna(text)):
        return text

    text = str(text).replace(">>", "")

    # Iteratively collapse repeated phrases, words, and character n-grams
    for pattern, replacement in [
        (r"\b((?:\S+\s+){1,5}\S+)(?:\s+\1){2,}\b", r"\1"),   # repeated phrases
        (r"\b(\S+)(?:\s+\1){2,}\b",                 r"\1"),   # repeated words
        (r"(.{2,}?)\1{2,}",                          r"\1"),   # repeated char-grams
    ]:
        prev = None
        while prev != text:
            prev = text
            text = re.sub(pattern, replacement, text)

    return re.sub(r"\s+", " ", text).strip()


def normalize_and_postprocess(text: str) -> str:
    """Apply Unicode normalization then hallucination-removal to a transcript."""
    return clean_transcript(normalize_bn_text(text))


print("✅ Text normalization utilities ready.")

# MODEL LOADING

- Loads the merged Whisper model directly from HuggingFace Hub.
- Optionally enables static KV cache and torch.compile.

In [ ]:
torch.cuda.empty_cache(); gc.collect()

print(f"📥 Loading processor and model : {MERGED_MODEL}")

processor = WhisperProcessor.from_pretrained(MERGED_MODEL)

model = WhisperForConditionalGeneration.from_pretrained(
    MERGED_MODEL,
    torch_dtype    = torch.float16 if USE_FP16 else torch.float32,
    low_cpu_mem_usage = True,
)

model = model.to(DEVICE)
model.eval()

# ── Static KV Cache ───────────────────────────────────────────────────────────
if USE_STATIC_CACHE:
    model.generation_config.cache_implementation = "static"

# ── torch.compile ─────────────────────────────────────────────────────────────
# The first inference call will be slow while the model is compiled;
# subsequent calls benefit from fused CUDA kernels.
if USE_TORCH_COMPILE and hasattr(torch, "compile") and DEVICE == "cuda":
    try:
        model = torch.compile(model, mode="reduce-overhead")
        print("   torch.compile  : enabled (first call will be slower)")
    except Exception as e:
        print(f"   torch.compile  : skipped ({e})")

print(f"✅ Model ready  |  device : {DEVICE}  |  dtype : {next(model.parameters()).dtype}")

if DEVICE == "cuda":
    alloc = torch.cuda.memory_allocated(0) / 1e9
    resv  = torch.cuda.memory_reserved(0)  / 1e9
    print(f"   VRAM — allocated : {alloc:.2f} GB  |  reserved : {resv:.2f} GB")

# TRANSCRIPTION ENGINE

In [ ]:
def transcribe_chunks_batch(
    audio_chunks: List[torch.Tensor],
    processor: WhisperProcessor,
    model: WhisperForConditionalGeneration,
    device: str,
    use_fp16: bool,
    batch_size: int,
    use_static_cache: bool,
) -> List[str]:
    """Run batched inference over a list of fixed-length audio chunks.

    Processes chunks in groups of `batch_size`, accumulates decoded strings,
    and clears the CUDA cache periodically to prevent memory fragmentation.
    """
    all_transcriptions = []

    for i in tqdm(range(0, len(audio_chunks), batch_size), desc="Batches", leave=False):
        batch = audio_chunks[i : i + batch_size]
        try:
            inputs = processor(
                [chunk.numpy() for chunk in batch],
                sampling_rate = 16000,
                return_tensors = "pt",
                padding = True,
            )
            input_features = inputs.input_features.to(device)
            if use_fp16:
                input_features = input_features.half()

            gen_kwargs = {
                "max_length": 448,
                "num_beams":  4,
                "do_sample":  False,
                "use_cache":  True,
            }
            if use_static_cache:
                gen_kwargs["cache_implementation"] = "static"

            with torch.no_grad():
                predicted_ids = model.generate(input_features, **gen_kwargs)

            decoded = processor.batch_decode(predicted_ids, skip_special_tokens=True)
            all_transcriptions.extend([t.strip() for t in decoded])

            # Periodic CUDA cache flush to reduce fragmentation
            if device == "cuda" and i % (batch_size * 4) == 0:
                torch.cuda.empty_cache()

        except Exception as e:
            print(f"   ⚠️  Batch error at chunk {i}: {e}")
            all_transcriptions.extend([""] * len(batch))

    return all_transcriptions


def transcribe_audio_file(
    audio_path: str,
    processor: WhisperProcessor,
    model: WhisperForConditionalGeneration,
    device: str,
    use_fp16: bool,
    chunk_length_s: int,
    overlap_s: float,
    batch_size: int,
    use_static_cache: bool,
) -> Dict:
    """End-to-end transcription for a single audio file.

    Pipeline: load → chunk → batch inference → merge → normalize & clean.

    Returns a dict with filename, transcript, duration, chunk count,
    processing time, and the original file path.
    """
    name       = Path(audio_path).name
    start_time = time.time()

    # Load & chunk
    audio, sr = load_and_resample_audio(audio_path)
    duration  = len(audio) / sr
    chunks    = chunk_audio(audio, sr, chunk_length_s, overlap_s)

    print(f"   {name}  |  {format_time(duration)}  |  {len(chunks)} chunks")

    # Inference
    raw_transcriptions = transcribe_chunks_batch(
        chunks, processor, model, device, use_fp16, batch_size, use_static_cache
    )

    # Merge and clean
    transcript = normalize_and_postprocess(merge_transcriptions(raw_transcriptions))
    elapsed    = time.time() - start_time

    print(f"   ✓ Done in {format_time(elapsed)}  |  speed {duration/elapsed:.2f}× realtime  |  {len(transcript)} chars")

    return {
        "filename":           Path(audio_path).stem,
        "transcript":         transcript,
        "duration_s":         round(duration, 2),
        "num_chunks":         len(chunks),
        "processing_time_s":  round(elapsed, 2),
        "audio_path":         audio_path,
    }


def process_multiple_files(
    audio_files: List[str],
    base_path: str,
    output_csv: str,
    processor: WhisperProcessor,
    model: WhisperForConditionalGeneration,
    device: str,
    use_fp16: bool,
    chunk_length_s: int,
    overlap_s: float,
    batch_size: int,
    use_static_cache: bool,
) -> pd.DataFrame:
    """Transcribe a list of audio files and write results to CSV.

    If AUDIO_FILES is empty, discovers all audio files in base_path automatically.
    Missing files are reported and skipped. Errors per file are caught and logged
    without stopping the overall pipeline.
    """
    # ── Resolve file list ──────────────────────────────────────────────────────
    if not audio_files:
        # Auto-discover all audio files in the directory
        audio_files = [
            p.name for p in Path(base_path).iterdir()
            if p.suffix.lower() in (".wav", ".mp3", ".flac", ".ogg", ".m4a")
        ]
        print(f"   Auto-discovered {len(audio_files)} audio file(s).")

    valid_paths, missing = validate_audio_files(audio_files, base_path)

    if missing:
        print(f"   ⚠️  {len(missing)} file(s) not found and will be skipped.")

    if not valid_paths:
        print("❌ No valid audio files to process.")
        return pd.DataFrame()

    # ── Process each file ──────────────────────────────────────────────────────
    results     = []
    total_start = time.time()

    for i, audio_path in enumerate(valid_paths, 1):
        print(f"\n[{i}/{len(valid_paths)}]")
        try:
            result = transcribe_audio_file(
                str(audio_path), processor, model, device, use_fp16,
                chunk_length_s, overlap_s, batch_size, use_static_cache,
            )
            results.append(result)
        except Exception as e:
            import traceback
            print(f"   ❌ Failed: {e}")
            traceback.print_exc()
            results.append({
                "filename":          audio_path.stem,
                "transcript":        f"[ERROR: {e}]",
                "duration_s":        0,
                "num_chunks":        0,
                "processing_time_s": 0,
                "audio_path":        str(audio_path),
            })
        finally:
            gc.collect()
            if device == "cuda":
                torch.cuda.empty_cache()

    # ── Write output CSV ───────────────────────────────────────────────────────
    df = pd.DataFrame(results)
    Path(output_csv).parent.mkdir(parents=True, exist_ok=True)
    df[["filename", "transcript"]].to_csv(output_csv, index=False, encoding="utf-8-sig")

    total_elapsed = time.time() - total_start
    print(f"\n✅ {len(results)} file(s) processed in {format_time(total_elapsed)}")
    print(f"   Output → {output_csv}")

    return df


print("✅ Transcription engine ready.")

# RUN INFERENCE

In [ ]:
inference_start = time.time()
print("=" * 65)
print("🚀  INFERENCE STARTED")
print(f"    {datetime.now(ZoneInfo('Asia/Dhaka')).strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 65)

df_results = process_multiple_files(
    audio_files      = AUDIO_FILES,
    base_path        = AUDIO_BASE_PATH,
    output_csv       = OUTPUT_CSV,
    processor        = processor,
    model            = model,
    device           = DEVICE,
    use_fp16         = USE_FP16,
    chunk_length_s   = CHUNK_LENGTH_S,
    overlap_s        = OVERLAP_S,
    batch_size       = BATCH_SIZE,
    use_static_cache = USE_STATIC_CACHE,
)

total_time = time.time() - inference_start
print("\n" + "=" * 65)
print("🏁  INFERENCE COMPLETE")
print(f"    Total time : {format_time(total_time)}")
print("=" * 65)

# RESULTS PREVIEW

In [ ]:
if not df_results.empty:
    print(f"\n{'FILE':<40} {'CHARS':>6}  PREVIEW")
    print("─" * 90)
    for _, row in df_results.iterrows():
        preview = str(row["transcript"])[:80].replace("\n", " ")
        print(f"  {row['filename']:<38} {len(str(row['transcript'])):>6}  {preview}…")
    print(f"\n✨ Results saved → {OUTPUT_CSV}")
else:
    print("⚠️  No results to display.")